In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
from tqdm.notebook import tqdm

sns.set(style="whitegrid")
train = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/train.csv")
test = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/test.csv")
all_df = pd.concat([train, test]).reset_index(drop=True)

display(train.head())
display(test.head())
display(all_df.head())

In [ ]:
print(len(train.eeg_id.unique()))
print(len(train.spectrogram_id.unique()))
print(len(train.patient_id.unique()))
print(len(train.eeg_id.unique())/len(train.patient_id.unique()))
print(len(train.spectrogram_id.unique())/len(train.patient_id.unique()))

In [ ]:

train_spectrogram_dir = '/kaggle/input/hms-harmful-brain-activity-classification/train_spectrograms/'
train_spectrogram_files = os.listdir(train_spectrogram_dir)
print(f'There are {len(train_spectrogram_files)} train spectrogram parquets')
test_spectrogram_dir = '/kaggle/input/hms-harmful-brain-activity-classification/test_spectrograms/'
test_spectrogram_files = os.listdir(test_spectrogram_dir)
print(f'There are {len(test_spectrogram_files)} test spectrogram parquets')
train_eeg_dir = '/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/'
train_eeg_files = os.listdir(train_eeg_dir)
print(f'There are {len(train_eeg_files)} train eeg parquets')
test_eeg_dir = '/kaggle/input/hms-harmful-brain-activity-classification/test_eegs/'
test_eeg_files = os.listdir(test_eeg_dir)
print(f'There are {len(test_eeg_files)} test eeg parquets')

In [ ]:
def get_files_info(files, file_dir):
    nan_ratio = []
    shapes = []
    for file in tqdm(files):    
        data = np.array(pd.read_parquet(f"{file_dir}{file}"))
        nan_ratio.append(np.isnan(data).sum() / len(data.flatten()))
        shapes.append(data.shape)
    nan_ratio = np.array(nan_ratio)
    shapes = np.array(shapes)
    return nan_ratio, shapes

In [ ]:
# train_spectrogram_nan_ratio, train_spectrogram_shapes = get_files_info(train_spectrogram_files, train_spectrogram_dir)
test_spectrogram_nan_ratio, test_spectrogram_shapes = get_files_info(test_spectrogram_files, test_spectrogram_dir)
# train_eeg_nan_ratio, train_eeg_shapes = get_files_info(train_eeg_files, train_eeg_dir)
test_eeg_nan_ratio, test_eeg_shapes = get_files_info(test_eeg_files, test_eeg_dir)

# print(train_spectrogram_nan_ratio.mean())
print(test_spectrogram_nan_ratio.mean())
# print(train_eeg_nan_ratio.mean())
print(test_eeg_nan_ratio.mean())
# print(np.unique(train_spectrogram_shapes))
print(np.unique(test_spectrogram_shapes))
# print(np.unique(train_eeg_shapes))
print(np.unique(test_eeg_shapes))

# 
# 
# 0.02423864616555035
# 0.0
# 0.0016409762644298224
# 0.0

In [ ]:
hypothesis0  = False
hypothesis1  = False

if test_spectrogram_nan_ratio.mean()<0.025:
    hypothesis0 = True    
    hypothesis1  = True
if test_spectrogram_nan_ratio.mean()<0.02:
    hypothesis0 = True    
    hypothesis1  = False
if test_spectrogram_nan_ratio.mean()<0.015:
    hypothesis0 = False    
    hypothesis1  = True
    
print(f'hypothesis0: {hypothesis0}')
print(f'hypothesis1: {hypothesis1}')

In [ ]:

hypotheses=[]
#there are no overlap of eeg ids in test set
hypotheses.append(len(test.eeg_id.unique()) == len(test))
#there are no overlap of spectrogram ids in test set
hypotheses.append(len(test.spectrogram_id.unique()) == len(test))
#there are overlap of patient ids in test set
hypotheses.append(len(test.patient_id.unique()) != len(test))
#eeg ids in test csv is the same as those in test_eeg dir
hypotheses.append(len(test_eeg_files) == len(test))
#spectrogram ids in test csv is the same as those ids in test_spectrogram dir
hypotheses.append(len(test_spectrogram_files) == len(test))
#number of spectrogram per patient < 6
hypotheses.append(len(train.spectrogram_id.unique())/len(train.patient_id.unique()) < 6)
#number of spectrogram per patient > 5
hypotheses.append(len(train.spectrogram_id.unique())/len(train.patient_id.unique()) > 5)
#eeg_ids of train and test set do not overlap
hypotheses.append(len(all_df.eeg_id.unique())==(len(train.eeg_id.unique())+len(test.eeg_id.unique())))
#spectrogram_ids of train and test set do not overlap
hypotheses.append(len(all_df.spectrogram_id.unique())==(len(train.spectrogram_id.unique())+len(test.spectrogram_id.unique())))
#patient_ids of train and test set do not overlap
hypotheses.append(len(all_df.patient_id.unique())==(len(train.patient_id.unique())+len(test.patient_id.unique())))
#shape of eeg is [20,10000]
hypotheses.append(np.array_equal(np.unique(test_eeg_shapes),np.array([20,10000])))
#shape of spectrogram is [300, 401]
hypotheses.append(np.array_equal(np.unique(test_spectrogram_shapes),np.array([300, 401])))
#num nan/data in test spectrogram < 0.03
hypotheses.append(test_spectrogram_nan_ratio.mean()<0.03)
#num nan/data in test spectrogram < 0.01
hypotheses.append(test_spectrogram_nan_ratio.mean()>0.01)
#there are no nan in eeg data
hypotheses.append(test_eeg_nan_ratio.mean()==0)

print(f'hypotheses: {hypotheses}')
hypotheses = all(hypotheses)
print(f'hypotheses: {hypotheses}')

In [ ]:
sub = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/sample_submission.csv")


if (hypothesis0 == True)&(hypothesis1 == True)&(hypotheses == True):
    # LB: 1.1
    mean_vote_ratio = {'seizure_vote':  0.196002,
    'gpd_vote':0.156386,
    'lrda_vote': 0.155805,
    'other_vote': 0.17610,
    'grda_vote': 0.17660,
    'lpd_vote': 0.139101}
elif (hypothesis0 == True)&(hypothesis1 == False)&(hypotheses == True):
    # LB: 1.0
    mean_vote_ratio = {'seizure_vote':    0.174031,
    'lpd_vote':        0.112700,
    'gpd_vote':        0.090854,
    'lrda_vote':       0.071484,
    'grda_vote':       0.136408,
    'other_vote':      0.414523,}
elif (hypothesis0 == False)&(hypothesis1 == True)&(hypotheses == True):
    # LB: 0.97
    mean_vote_ratio = {'seizure_vote':    0.152810,
    'lpd_vote':        0.142456,
    'gpd_vote':        0.104062,
    'lrda_vote':       0.065407,
    'grda_vote':       0.114851,
    'other_vote':      0.420414,}
elif (hypothesis0 == False)&(hypothesis1 == False)&(hypotheses == True):
    # LB: 1.28
    mean_vote_ratio = {'seizure_vote': 0.310718,
    'lpd_vote': 0.046279,
    'gpd_vote': 0.051885,
    'lrda_vote': 0.081796,
    'grda_vote': 0.231471,
    'other_vote': 0.277851,}
else:
    # sub will fail
    mean_vote_ratio = {'seizure_vote': np.nan,
    'lpd_vote': np.nan,
    'gpd_vote': np.nan,
    'lrda_vote': np.nan,
    'grda_vote': np.nan,
    'other_vote': np.nan,}


targets = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
for target in targets:
    sub[target] = mean_vote_ratio[target]
sub.head()

In [ ]:
sub.to_csv("/kaggle/working/submission.csv", index=False)